# Batthleship AI GameBot Workshop #1

### What is a library?

A library is a collection of pre-written code that can be imported into a workspace that has functions and classes that we can use to perform specific operations. In the data science workshop, we imported the NumPy library to perform arithmetic operations on arrays and the Pandas library to import and work with large datasets.

In the workshop today we will import the "random" and "pygame" libraries. They each provide specific functions and classes to help us build our gamebot:

random: functions and classes to generate psuedo-random numbers and make random selections <br>
pygame: functions and classes to create and render graphics, sounds, and user input for 2D games

In [30]:
pip install pygame

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [31]:
# code to import libraries
import random
import pygame

## Pygame Initialization and Global Variables

In this section, we will initialize Pygame and define the essential global variables and constants that will control the game's display, visual elements, and gameplay settings. This setup includes configuring the screen dimensions, colors, grid sizes, and other parameters that define the look and feel of the game.



In [32]:
# Pygame initialization
pygame.init()
pygame.display.set_caption("Battleship")

### What is a global variable?

A global variable is a variable declared outside of any functions so that it can be accessed or modified at any part of the program--its scope is global.

In the next step we are going to define several **global constant variables** which we will define to create our display window where we will see our game rendered. A global constant variable after being defined is not modified, thus it is constant. We write global constant variable names in all caps. Take two-three minutes to think of what variables we will need for our game display window.

In [33]:
# Global variables
SQ_SIZE = 40
H_MARGIN = SQ_SIZE * 4
V_MARGIN = SQ_SIZE
WIDTH = SQ_SIZE * 10 * 2 + H_MARGIN
HEIGHT = SQ_SIZE * 10 * 2 + V_MARGIN
INDENT = 5

### Colors: RGB Values

We next create global variables for colors for different elements of our game and the display. Take a couple minutes to read [this article](https://www.w3schools.com/html/html_colors_rgb.asp) about we specify colors using RGB values, and then fill in the values for color variables below.

In [34]:
# Colors
GREY = (170, 170, 170)  # RGB value for the background color
WHITE = (255, 255, 255)  # RGB value for the white color
GREEN = (131, 228, 94)  # RGB value for the green color (possibly for highlighting or other elements)
BLUE = (51, 164, 255)  # RGB value for the blue color (for misses)
RED = (196, 30, 58)  # RGB value for the red color (for sunk ships or hits)
PINK = (233, 158, 193)  # RGB value for the pink color (for hits)

### Creating a Mapping for Different Game States

We will use a dictionary to create a mapping of the different game states (background, miss, hit, sink) on our display using our color variables that we just defined. Take a couple minutes to read about how [dictionaries](https://www.w3schools.com/python/python_dictionaries.asp) function and see if you can map to the four different states we would have.

In [35]:
# Color mapping for game states
COLORS = {"U":GREY, "M":BLUE, "H":PINK, "S": RED}

### Initialize Pygame Screen

Finally, let's create a `SCREEN` variable using the pygame `display.set_mode` function. Look at the documentation for this function and see if you can figure out what variables we previously defined to pass in to the function as parameters.

Hint: You only need to fill in the `size` parameter.

In [36]:
# Initialize the Pygame screen
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))

## Drawing the Game Grids

In this section, we will draw the game grids for both the player and the opponent. The game board is made up of two 10x10 grids, each represented by a series of rectangles. These grids serve as the interface where players can interact and track their progress.

When the `search` flag is enabled, additional feedback will be shown on the grids, such as indicating where hits or misses have occurred. This visual feedback helps players track their moves and make strategic decisions throughout the game.




### What is a function?

A function is a block of code designed to perform a specific task. In python, we create a function using the keyword `def`, followed by the function name, parentheses, and a colon. Within our parantheses, we can specify arguments that need to be passed in when the function is called, and we can also set default parameters, so if the function is called and parameters aren't passed in, there are values for the function to default to.

To call a function, we simply write the function name with parantheses and pass in any arguments that may be needed.


In [37]:
def draw_grid(player, left=0, top=0, search=False):
    """
    This function draws the grid for our battleship board.
    """
    for i in range(100):  # Loop through each of the 100 grid squares
        # Column that we are in (horizontal position)
        x = left + i % 10 * SQ_SIZE # Calculate the x position based on the column index
        # Row that we are in (vertical position)
        y = top + i // 10 * SQ_SIZE # Calculate the y position based on the row index
        
        square = pygame.Rect(x, y, SQ_SIZE, SQ_SIZE)  # Create a rectangle for the square
        pygame.draw.rect(SCREEN, WHITE, square, width=3)  # Draw the grid square
        
        if search:  # If the search flag is enabled
            x += SQ_SIZE // 2  # Adjust x position for the circle (centered in the square)
            y += SQ_SIZE // 2  # Adjust y position for the circle (centered in the square)
            
            pygame.draw.circle(SCREEN, COLORS[player.search[i]], (x, y), radius=(SQ_SIZE // 4)) # Draw a circle for the hit/miss indicator

## Creating our ships

We’ll begin by creating the game board and ships. Each ship will have a random position and orientation on the grid. Then, we will code the gameplay mechanics, allowing players to take turns guessing and trying to sink each other's ships.

Ready to get started? Let’s dive into the code and start building our Battleship game!

Before diving into the full game, we will first look into how we can model a **ship** in our game. A ship in the Battleship game has several important features:

- **Size**: How many spaces the ship occupies on the grid.
- **Position**: The starting position of the ship, which will be randomly selected on the grid.
- **Orientation**: Whether the ship is placed horizontally (across rows) or vertically (across columns).
- **Indexes**: The list of grid positions that the ship occupies.

We will create a **Ship class** to define these properties. The class will include methods to randomly place the ship on the grid, taking into account its size and orientation. Understanding this class will help us in creating and managing ships during the game.

In [38]:
class Ship:
    def __init__(self, size):  # Constructor to initialize a ship with a given size
        
        self.row = random.randrange(0, 9)  # Random row position (between 0 and 9)
        self.col = random.randrange(0, 9)  # Random column position (between 0 and 9)
        
        self.size = size  # The size of the ship (number of grid squares it occupies)
        
        self.orientation = random.choice(["h", "v"])  # Randomly choose the orientation: "h" for horizontal, "v" for vertical
        
        self.indexes = self.compute_indexes()  # Call method to compute the indexes of the ship's positions
        
    def compute_indexes(self):  # Compute the grid indexes where the ship will be placed
        start_index = self.row * 10 + self.col  # Starting index based on row and column
        if self.orientation == 'h':  # If the ship is placed horizontally
            return [start_index + i for i in range(self.size)]  # Generate horizontal positions
        elif self.orientation == "v":  # If the ship is placed vertically
            return [start_index + i * 10 for i in range(self.size)]  # Generate vertical positions


# Drawing Ships on a Grid
 
**Before diving into the code, let’s visualize the grid**

- Imagine a chessboard where each square represents a possible position for a ship.

- Each square has a fixed size, defined by SQ_SIZE (e.g., 50 pixels).

- Ships can be placed horizontally (left to right) or vertically (top to bottom).

The grid is like a map, and the ships are like objects placed on that map. Our goal is to draw these ships in the correct positions.
In this code, we are creating a function called `draw_ships` that will draw ships onto a grid. This grid represents a player's game board, and the ships are placed on this board. Let's break down the code step by step to understand how it works.

In [39]:
# Function to draw ships onto the position grids
def draw_ships(player, left=0, top=0):
    for ship in player.ships:  # Loop through each ship in the player's list of ships
        x = left + ship.col * SQ_SIZE + INDENT  # Calculate the x position for the ship
        y = top + ship.row * SQ_SIZE + INDENT  # Calculate the y position for the ship
        
        if ship.orientation == "h":  # If the ship is horizontal
            width = ship.size * SQ_SIZE - 2 * INDENT  # The width of the ship (size times square size, minus margins)
            height = SQ_SIZE - 2 * INDENT  # The height of the ship (equal to the size of one square, minus margins)
        else:  # If the ship is vertical
            width = SQ_SIZE - 2 * INDENT  # The width of the ship (equal to the size of one square, minus margins)
            height = ship.size * SQ_SIZE - 2 * INDENT  # The height of the ship (size times square size, minus margins)
        
        rectangle = pygame.Rect(x, y, width, height)  # Create a rectangle for the ship's area
        pygame.draw.rect(SCREEN, GREEN, rectangle, border_radius=15)  # Draw the rectangle with rounded corners

## The Player Class

The **Player class** encapsulates everything related to a player in the game. It tracks critical game elements such as the player's ships, their positions on the grid, and the state of the player’s search grid (which records where the player has targeted and whether they've hit or missed enemy ships). Below is a detailed breakdown of the class and its key components.

### Key Responsibilities of the Player Class:
1. **Managing Ships:** Each player has a set of ships placed on the grid. The class stores information about the ships' positions and sizes.
2. **Tracking Search Grid:** The class maintains a grid that records the player's attempts to search for enemy ships. This grid is updated with hits, misses, or sunk ships.
3. **Ship Sinking Logic:** It tracks whether a ship has been sunk based on the player's search grid and updates the status accordingly.


In [40]:
class Player:
    def __init__(self):
        self.ships = []
        self.search = ["U" for i in range(100)] #U for unknown 
        self.place_ships(sizes = {5, 4, 3, 3, 2})
        list_of_lists = [ship.indexes for ship in self.ships]
        self.indexes = [index for sublist in list_of_lists for index in sublist]
        
    def place_ships(self, sizes):
        for size in sizes:
            placed = False
            while not placed:
                #create a new ship
                ship = Ship(size)
                
                #check if placement is possible
                possible = True
                for i in ship.indexes: 
                    #indexes must be < 100:
                    if i >= 100:
                        possible = False
                        break
                        
                    #ships cannot behave like snakes
                    new_row = i // 10
                    new_col = i % 10
                    if new_row != ship.row and new_col != ship.col:
                        possible = False 
                        break 
                    
                    #ships cannot intersect   
                    for other_ship in self.ships:
                        if i in other_ship.indexes:
                            possible = False
                            break
                #place the ship            
                if possible:
                    self.ships.append(ship)
                    placed = True
                    
    def show_ships(self):
        indexes = ["-" if i not in self.indexes else "X" for i in range(100)]
        for row in range(10):
            print(" ".join(indexes[(row-1)*10: row*10]))

## Game Class: Handling the Game Logic

The `Game` class is responsible for managing the overall flow of the game, tracking player turns, making moves, and determining the game state. Below is a breakdown of the key components of the class.

### 1. **Initialization (`__init__` method):**

   - **Player Instances:**
     - `self.player1 = Player()` and `self.player2 = Player()` create instances of the `Player` class, which represent the two players in the game. These instances will hold each player's ships, search grid, and other game-related information.

   - **Player Turn Tracking:**
     - `self.player1_turn = True` is a boolean variable that keeps track of whose turn it is. Initially, Player 1 is set to take the first turn.

   - **Game Over and Result Flags:**
     - `self.over = False` tracks whether the game is over.
     - `self.result = None` is used to store the result of the game (e.g., who won).

### 2. **Making a Move (`make_move` method):**

   The `make_move` method processes a player's move by updating the search grid and checking for hits, misses, and sunk ships.

   - **Determine Active Player and Opponent:**
     - The current player is determined based on the `player1_turn` boolean: `player = self.player1 if self.player1_turn else self.player2`.
     - The opponent is also determined similarly: `opponent = self.player2 if self.player1_turn else self.player1`.

   - **Hit or Miss:**
     - The `hit = False` flag is initialized but isn't used directly here.
     - If the selected square (`i`) is in the opponent's list of ship indexes, it is marked as a hit (`"H"`) on the player's search grid.
     - If the selected square is not part of the opponent's ships, it is marked as a miss (`"M"`) on the player's search grid.

   - **Ship Sinking Check:**
     - If a hit occurs, the code checks if any of the opponent's ships are sunk. A ship is considered sunk if all of its indexes are marked as hits (`"H"`).
     - If a ship is sunk, all its indexes are updated to `"S"` (sunk).

   - **Switch Turn:**
     - After processing the move, the active player's turn is switched by toggling the `player1_turn` boolean: `self.player1_turn = not self.player1_turn`.


The `Game` class serves as the central controller for handling player actions and determining the game's progression. It manages alternating turns, updates the game state (hit, miss, sunk ships), and eventually determines when the game is over (though the `over` flag and result calculation are not implemented in this snippet).



In [41]:
class Game:
    def __init__(self):
        
        #These are instances of the Player class, representing the two players in the game.
        self.player1 = Player() 
        self.player2 = Player()
        
        #This boolean variable keeps track of whose turn it is
        self.player1_turn = True
        
        self.over = False
        self.result = None
   
    #defining whose move it is    
    def make_move(self, i):
        player = self.player1 if self.player1_turn else self.player2
        opponent = self.player2 if self.player1_turn else self.player1
        hit = False
        
        #set miss "M" or hit "H"
        if i in opponent.indexes:
            player.search[i] = "H"
            
            #check if ship sunk
            for ship in opponent.ships:
                sunk = True
                for i in ship.indexes:
                    if player.search[i] == "U":
                        sunk = False
                        break
                if sunk:
                    for i in ship.indexes:
                        player.search[i] = "S"
                        break

        #if the guess is a miss
        else:
            player.search[i] = "M"
            
        # change the active player 
        self.player1_turn = not self.player1_turn

## Game Loops in Action

In game development, the **game loop** is the heart of the game. It repeatedly handles user inputs, updates the game state, and renders the visuals on the screen. The loop continues until the game ends, ensuring an interactive experience.

### 1. **Process Input:**

   The game loop starts by processing user input, which is usually done by listening to events. In the provided code, this happens in the `for event in pygame.event.get()` loop. The input processed here includes:
   
   - **Mouse Clicks (`MOUSEBUTTONDOWN`):** The user can click on the game grid to make a move. Based on the coordinates of the mouse click, the game decides whether the player is interacting with their own grid or their opponent's grid.
   - **Key Presses (`KEYDOWN`):** The game listens for keyboard events such as:
     - **Escape Key (`K_ESCAPE`)** to close the game.
     - **Space Bar (`K_SPACE`)** to pause or unpause the game.

   The input here triggers updates to the game state, such as changing whose turn it is or processing the player’s move.

### 2. **Update Game State:**

   Once the input is processed, the game state needs to be updated based on the input and the rules of the game. This is done in the `make_move` method, which updates the search grid of the current player, marks hits and misses, checks if any ships are sunk, and switches the turn between the two players.
   
   - **Making a Move:** The player’s move is processed (hit or miss), and the game checks if any of the opponent's ships are sunk after a hit.
   - **Switching Turns:** After a move is made, the game switches the active player, making the game alternate between Player 1 and Player 2.

   The game state is updated by altering the `search` grid of each player (either marking it with `"H"` for hit, `"M"` for miss, or `"S"` for sunk).

### 3. **Render:**

   After the game state is updated, the next step is to render the game state to the screen, making the game visually interactive. In the provided code, this is done in the following lines:

   - **Draw Background:** The screen is filled with a grey background (`SCREEN.fill(GREY)`).
   - **Draw Grids:** The search grids of both players are drawn in their respective locations. Each player has a search grid and a position grid.
   - **Draw Ships:** Ships are drawn on the position grids to represent where the ships are placed for each player.
   - **Update the Screen:** After everything is drawn, `pygame.display.flip()` updates the screen to show the latest changes.

### 4. **Loop Continuation:**

   These steps (processing input, updating the game state, and rendering) are executed repeatedly inside the game loop until the game ends. The loop will continue to run as long as the game is not paused or closed, ensuring a continuous and dynamic gameplay experience.



In [42]:
game = Game() #call our Game class from before                           

# Pygame loop
animating = True
pausing = False

## Pygame Animation with User Interaction

### Event Handling
The code begins by checking for different types of events during the animation loop:

#### 1. **Handling Window Closure:**
   - The program listens for the `QUIT` event, which happens when the user closes the window. If this event is triggered, the `animating` variable is set to `False`, stopping the animation.

#### 2. **Handling Mouse Clicks:**
   - The program also listens for mouse button events. When the mouse is clicked (`MOUSEBUTTONDOWN`), it tracks the mouse's coordinates (`x`, `y`).
   - Based on the player's turn, it determines if the mouse click is within the player's grid:
     - **Player 1's Grid:** The click is checked to ensure it falls within the bounds of Player 1’s search grid (top left corner).
     - **Player 2's Grid:** Similarly, the click is checked for Player 2’s grid (bottom right corner).
   - After determining which grid is clicked, the program calculates the row and column of the clicked square, then converts the position into an index using the formula: `index = row * 10 + col`. This index is then passed to `game.make_move()` to update the game state.

#### 3. **Handling Keyboard Events:**
   - The program also listens for key presses, such as:
     - **Escape Key (`K_ESCAPE`)**: When pressed, the animation stops by setting `animating = False`.
     - **Space Bar (`K_SPACE`)**: Toggles the `pausing` state between `True` and `False`, which pauses or unpauses the game.

### Animation and Drawing
If the game is not paused (`pausing = False`), the animation continues:

#### 1. **Drawing the Background:**
   - The screen is filled with a grey background color: `SCREEN.fill(GREY)`.

#### 2. **Drawing the Grids:**
   - The grids for both players are drawn in different sections of the screen:
     - **Player 1's Search Grid:** The search grid for Player 1 is drawn at the top-left corner.
     - **Player 2's Search Grid:** The search grid for Player 2 is drawn at the bottom-right corner.
   - **Player 1's Position Grid:** This grid is drawn at the top-right corner.
   - **Player 2's Position Grid:** This grid is drawn at the bottom-left corner.

#### 3. **Drawing Ships:**
   - The ships for both players are drawn on their respective position grids:
     - **Player 1's Ships:** The ships are drawn on Player 1’s position grid (top-right).
     - **Player 2's Ships:** The ships are drawn on Player 2’s position grid (bottom-left).

#### 4. **Updating the Screen:**
   - After everything is drawn, the screen is updated using `pygame.display.flip()`, which updates the display to reflect the latest changes.

This loop runs continuously, handling user inputs and updating the game state. The animation continues until the user closes the window or pauses the game.



In [43]:
while animating:
    # To track user interaction
    for event in pygame.event.get():
        
        # user closes the window                                      
        if event.type == pygame.QUIT:
            animating = False  # Set this to False when user closes the window
                                              
        # user clicks on mouse
        if event.type == pygame.MOUSEBUTTONDOWN:
            # Get mouse position
            x, y = pygame.mouse.get_pos()
            
            # If it's Player 1's turn and the mouse is in the correct area
            if game.player1_turn and x < SQ_SIZE * 10 and y < SQ_SIZE * 10: 
                # Calculate row and column based on mouse position
                row = y // SQ_SIZE  # Calculate row based on the y position and SQ_SIZE
                col = x // SQ_SIZE  # Calculate column based on the x position and SQ_SIZE
                index = row * 10 + col  # Calculate index based on row and col
                game.make_move(index)  # Call the make_move function for Player 1
                
            # If it's Player 2's turn and the mouse is in the correct area
            elif not game.player1_turn and x > WIDTH - SQ_SIZE * 10 and y > SQ_SIZE * 10 + V_MARGIN:
                # Calculate row and column based on mouse position
                row = (y - SQ_SIZE * 10 - V_MARGIN) // SQ_SIZE # Calculate row for Player 2
                col = (x - SQ_SIZE * 10 - H_MARGIN) // SQ_SIZE # Calculate column for Player 2
                index = row * 10 + col  # Calculate index for Player 2
                game.make_move(index)  # Call the make_move function for Player 2
                
        # User presses key on keyboard
        if event.type == pygame.KEYDOWN:
            # Escape key to close the animation
            if event.key == pygame.K_ESCAPE:
                animating = False  # Exit the game when the Escape key is pressed

            # Space bar to pause and unpause
            if event.key == pygame.K_SPACE:
                pausing = not pausing  # Toggle pausing state when Spacebar is pressed

    # If the game is not paused, continue with the animation
    if not pausing:
        # draw background
        SCREEN.fill(GREY)  # Fill the screen with a grey background

        # draw search grids for both players
        draw_grid(game.player1, search=True)  # Draw search grid for Player 1 (top left)
        draw_grid(game.player2, search=True, left=(WIDTH - H_MARGIN) // 2 + H_MARGIN, 
                  top=(HEIGHT - V_MARGIN) // 2 + V_MARGIN)  # Draw search grid for Player 2 (bottom right)
        
        # draw position grids for both players
        draw_grid(game.player1, left=((WIDTH - H_MARGIN) // 2 + H_MARGIN))  # Draw position grid for Player 1 (top right)
        draw_grid(game.player2, top=(HEIGHT - V_MARGIN) // 2 + V_MARGIN)  # Draw position grid for Player 2 (bottom left)
        
        # draw ships onto position grids for both players
        draw_ships(game.player1, top=(HEIGHT - V_MARGIN) // 2 + V_MARGIN)  # Draw ships for Player 1
        draw_ships(game.player2, left=((WIDTH - H_MARGIN) // 2 + H_MARGIN))  # Draw ships for Player 2
        
        # Update the screen with the new drawing
        pygame.display.flip()  # This updates the window with everything drawn so far
